In [6]:
import glob
import boto3
import os
import pandas as pd

bucket='sagemaker-us-east-1-719086491758'
prefix = "final_images" # replace this with the path to your images
CSVIN='twitter_with_LDA.csv'
CSVOUT='twitter_with_LDA_and_image_data.csv'

In [35]:
df = pd.read_csv(CSVIN)
# Convert these numbers to categories
df['id'] = df['id'].astype('object')
print(df.shape)
df.head()

(802, 29)


,id,created_at,retweets,likes,replies,quotes,user_followers,user_following,user_listed,user_tweets,...,Dominant_score,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,1374863679246659585,2021-03-24 23:20:19+00:00,0,0,0,0,5992,5887,318,205698,...,0.287116,0.012806,0.287116,0.120242,0.012807,0.012805,0.175035,0.012806,0.160291,0.206093
1,1374863677107671042,2021-03-24 23:20:19+00:00,0,0,0,0,137,854,1,1124,...,0.370377,0.370377,0.370356,0.037038,0.037038,0.037038,0.037038,0.037038,0.037038,0.037038
2,1374863674100183046,2021-03-24 23:20:18+00:00,0,0,0,0,62,164,0,4208,...,0.111186,0.111115,0.111067,0.111077,0.111132,0.111186,0.111074,0.111106,0.111094,0.111149
3,1374863668869918721,2021-03-24 23:20:17+00:00,11,0,0,0,40783,40417,469,104938,...,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
4,1374863657578946570,2021-03-24 23:20:14+00:00,45,0,0,0,718,1046,14,13761,...,0.555388,0.055604,0.055572,0.055577,0.055572,0.555388,0.055572,0.055572,0.055572,0.055572


In [36]:
#limit number of rows for testing-if you don't want to process everything
df=df.head(2)
#df.dtypes
df2 = df
df

,id,created_at,retweets,likes,replies,quotes,user_followers,user_following,user_listed,user_tweets,...,Dominant_score,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,1374863679246659585,2021-03-24 23:20:19+00:00,0,0,0,0,5992,5887,318,205698,...,0.287116,0.012806,0.287116,0.120242,0.012807,0.012805,0.175035,0.012806,0.160291,0.206093
1,1374863677107671042,2021-03-24 23:20:19+00:00,0,0,0,0,137,854,1,1124,...,0.370377,0.370377,0.370356,0.037038,0.037038,0.037038,0.037038,0.037038,0.037038,0.037038


In [10]:
#get the rekognition client
client = boto3.client('rekognition')

In [11]:
#get the s3 client
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)
#files = my_bucket.objects.filter(Prefix=prefix)

In [33]:
dfFaces = pd.DataFrame([])
i=0

for row in df.itertuples():
    file = f"{row.id}"
    extension = f"{row.url.split('.')[-1]}"
    print(f"Processing TweetID - {file}.{extension}")    
    if not pd.isnull(file):
        #print(i, row[1], file, f"{prefix}/{row[1]}.{file.split('.')[-1]}")
    
    
        if extension == 'jpg':
            # call rekognition for this next file
            response = client.detect_faces(
                Image={
                    'S3Object': {
                        'Bucket': bucket,
                        'Name': f"{prefix}/{file}.{extension}"
                    }
                },
                Attributes=[
                    'ALL',
                ]
            )

            # now add all of the facial features for every person found in the photo
            for fd in response["FaceDetails"]:    
                i=i+1
                dfFaces.loc[i,'TweetID'] = file
                dfFaces.loc[i,'PersonID'] = i
                dfFaces.loc[i,'AgeRange-Low'] = fd["AgeRange"]["Low"]
                dfFaces.loc[i,'AgeRange-High'] = fd["AgeRange"]["High"]
                dfFaces.loc[i,'Smile'] = fd["Smile"]["Value"]
                dfFaces.loc[i,'Gender'] = fd["Gender"]["Value"]
                dfFaces.loc[i,'Emotion'] = fd["Emotions"][0]["Type"] #get dominant emotion
                dfFaces.loc[i,'Emotion-Confidence'] = fd["Emotions"][0]["Confidence"] #get dominant emotion


Processing TweetID - 1374863677107671042.jpg


,id,created_at,retweets,likes,replies,quotes,user_followers,user_following,user_listed,user_tweets,...,topic_7,topic_8,topic_9,Count-People,Avg-AgeRange-Low,Avg-AgeRange-High,Count-Smile,Percent-Smile,HAPPY,Female
0,1374863677107671042,2021-03-24 23:20:19+00:00,0,0,0,0,137,854,1,1124,...,0.037038,0.037038,0.037038,2,1.0,22.0,1,1.0,2,2


In [13]:
dfFaces.describe

<bound method NDFrame.describe of                TweetID  PersonID  AgeRange-Low  AgeRange-High  Smile  Gender  \
1  1374863677107671042       1.0          12.0           22.0   True  Female   
2  1374863677107671042       2.0           1.0            7.0  False  Female   

  Emotion  Emotion-Confidence  
1   HAPPY           99.595222  
2   HAPPY           51.809086  >

In [14]:
dfFaceSummary = pd.DataFrame([])

#summarize stats per file
#aggregate functions include min, max, mean, count, and more.
dfFaceSummary['Count-People'] = dfFaces.groupby('TweetID')['PersonID'].count()
dfFaceSummary['Avg-AgeRange-Low'] = dfFaces.groupby('TweetID')['AgeRange-Low'].min()
dfFaceSummary['Avg-AgeRange-High'] = dfFaces.groupby('TweetID')['AgeRange-High'].max()
dfFaceSummary['Count-Smile'] = dfFaces[dfFaces['Smile']==True].groupby('TweetID')['Smile'].count()
dfFaceSummary['Count-Smile'] = dfFaceSummary['Count-Smile'].fillna(0)
#****Fix this****
dfFaceSummary['Percent-Smile'] = dfFaceSummary['Count-Smile']/1
dfFaceSummary['Percent-Smile'] = dfFaceSummary['Percent-Smile'].fillna(0)

dfFaceSummary = dfFaceSummary.merge(dfFaces.groupby('TweetID')['Emotion'].value_counts().unstack().fillna(0), on='TweetID')
dfFaceSummary = dfFaceSummary.merge(dfFaces.groupby('TweetID')['Gender'].value_counts().unstack().fillna(0), on='TweetID')

#print out 
dfFaceSummary

,Count-People,Avg-AgeRange-Low,Avg-AgeRange-High,Count-Smile,Percent-Smile,HAPPY,Female
TweetID,,,,,,,
1374863677107671042,2,1.0,22.0,1,1.0,2,2


In [15]:
#the tweet id is an object, so we need to convert it to the same data type as our id
dfFaceSummary.index = dfFaceSummary.index.astype(int)

#Now merge the frames together
df = df.merge(dfFaceSummary, left_on='id', right_on='TweetID')
df

,id,created_at,retweets,likes,replies,quotes,user_followers,user_following,user_listed,user_tweets,...,topic_7,topic_8,topic_9,Count-People,Avg-AgeRange-Low,Avg-AgeRange-High,Count-Smile,Percent-Smile,HAPPY,Female
0,1374863677107671042,2021-03-24 23:20:19+00:00,0,0,0,0,137,854,1,1124,...,0.037038,0.037038,0.037038,2,1.0,22.0,1,1.0,2,2


In [16]:
#now save this out to a new csv
df.to_csv(CSVOUT, index=False)

In [38]:
dflabels = pd.DataFrame([])

for row in df2.itertuples():
    file = f"{row.id}"
    extension = f"{row.url.split('.')[-1]}"
    print(f"Processing TweetID - {file}.{extension}")    
    if not pd.isnull(file):
        #print(i, row[1], file, f"{prefix}/{row[1]}.{file.split('.')[-1]}")
    
    
        if extension == 'jpg':
            # call rekognition for this next file
            response = client.detect_labels(
            Image={
                    'S3Object': {
                        'Bucket': bucket,
                        'Name': f"{prefix}/{file}.{extension}"
                    }
            }, MaxLabels=10
        )

        #now add all of the labels found in this photo
        for lb in response["Labels"]:
            print (lb["Name"] + " " + str(lb["Confidence"]))
            

Processing TweetID - 1374863679246659585.jpg
Fork 97.4509506225586
Plant 96.89115905761719
Food 89.39244842529297
Broccoli 84.70565032958984
Vegetable 84.70565032958984
Meal 83.43388366699219
Dish 72.7592544555664
Bowl 70.695068359375
Dining Table 61.07152557373047
Table 61.07152557373047
Processing TweetID - 1374863677107671042.jpg
Person 99.79623413085938
Eating 96.64192962646484
Food 96.64192962646484
Ice Cream 93.67967987060547
Meal 90.25479125976562
Dish 87.05123138427734
People 70.41609954833984
Hot Dog 60.81785583496094
Couch 60.0781135559082
Restaurant 56.7321891784668
